## Run the following command when launching for the first time

In [1]:
!pip install datasets transformers evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (

## Load dataset

Link to dataset: https://huggingface.co/datasets/kkotkar1/course-reviews

In [2]:
from datasets import load_dataset

dataset = load_dataset("kkotkar1/course-reviews")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Preprocess

model obtained from: https://huggingface.co/docs/transformers/model_doc/distilbert

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
#this tokenizes all the sentences
def preprocess(sentences):
  new_reviews = tokenizer(sentences["review"], truncation=True)
  return new_reviews

In [5]:
#this fixes the indexes of the labels
def preprocess_label(sentences):
  labels = sentences["label"]
  sentences["label"] = labels - 1
  return sentences

In [6]:
new_dataset = dataset.map(preprocess_label)
tokenized_reviews = new_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/80263 [00:00<?, ? examples/s]

Map:   0%|          | 0/26755 [00:00<?, ? examples/s]

Map:   0%|          | 0/80263 [00:00<?, ? examples/s]

Map:   0%|          | 0/26755 [00:00<?, ? examples/s]

In [7]:
#this is used to apply padding to the training batches while training

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length", max_length=512)

## Evaluate

In [8]:
import evaluate

accuracy = evaluate.load("accuracy")

In [9]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Train

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=5, n_layers=8
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'distilbert.transformer.layer.6.attention.k_lin.bias', 'distilbert.transformer.layer.6.attention.k_lin.weight', 'distilbert.transformer.layer.6.attention.out_lin.bias', 'distilbert.transformer.layer.6.attention.out_lin.weight', 'distilbert.transformer.layer.6.attention.q_lin.bias', 'distilbert.transformer.layer.6.attention.q_lin.weight', 'distilbert.transformer.layer.6.attention.v_lin.bias', 'distilbert.transformer.layer.6.attention.v_lin.weight', 'distilbert.transformer.layer.6.ffn.lin1.bias', 'distilbert.transformer.layer.6.ffn.lin1.weight', 'distilbert.transformer.layer.6.ffn.lin2.bias', 'distilbert.transformer.layer.6.ffn.lin2.weight', 'distilbert.transformer.layer.6.output_layer_norm.bias', 'distilbert.transformer.layer.6.output_layer_norm.weight', 'distilbert.transformer.layer.6.sa_layer_n

In [12]:
training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.05,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_reviews["train"],
    eval_dataset=tokenized_reviews["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("my_model_8")

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
model.save("trained_model")

## Inference

In [ ]:
text = "I really enjoyed the content of this class!"

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="./my_model_8")
classifier(text)

[{'label': 'LABEL_4', 'score': 0.7473529577255249}]